# Plan:

1. DNA
2. Genetic ALgorithm
3. Inclination of plane
4. all together.

In [49]:
import numpy as np

np.random.seed(42)

In [50]:
from mujoco_py import load_model_from_path, utils
from mujoco_py import MjSim, MjViewer, MjViewerBasic

In [51]:
import matplotlib.pyplot as plt

In [52]:
import os
#mj_path, _ = utils.discover_mujoco()

#xml_path = os.path.join(mj_path, 'model', 'ramp-v0.xml')
xml_path = os.path.join('./', 'model', 'one-ramp.xml')

In [53]:
xml_path

'./model/one-ramp.xml'

In [54]:
class DNA:
    Genes   = []
    BinaryGenes = []
    Fitness = 0.0
    
    # Significant digits for binary conversion
    sigDig = 4
    
    def __init__(self, size):
        self.Genes = [self.getRandomGene() for i in range(size)]
        self.BinaryGenes = [self.toBinary(self.Genes[i]) for i in range(size)]
        self.Fitness = 0.0
        #print(self.BinaryGenes)
        
    def getRandomGene(self, low=-1, high=1):
        # Returns random angle
        return np.random.uniform(low, high)
        
    def crossover(self, otherParent):
        size  = len(self.Genes)
        child = DNA(size)
        
        for i in range(size):
            childBinary = self.toBinary(child.Genes[i])
            
            #print(self.BinaryGenes)
            
            for j in range(len(childBinary)):
                if np.random.random() < 0.5:
                    #child.Genes[i] = self.Genes[i]
                    #print(self.BinaryGenes, i,j)
                    #print(childBinary, i,j)
                    childBinary[j] = self.BinaryGenes[i][j]
                else:
                    #child.Genes[i] = otherParent.Genes[i]
                    childBinary[j] = otherParent.BinaryGenes[i][j]
                    
            child.Genes[i] = self.fromBinary(childBinary)
        return child
    
    def setFitness(self, fitness):
        self.Fitness = fitness
        
    def mutate(self, rate):
        for i in range(len(self.Genes)):
            for j in range(len(self.BinaryGenes[i])):
                if np.random.random() < rate:
                    self.BinaryGenes[i][j] = 1 - self.BinaryGenes[i][j]
            self.Genes[i] = self.fromBinary(self.BinaryGenes[i])
    
    def toBinary(self, num, bits=16, sigDig=4):
        rest = int(num*10**sigDig)
        binary = []
        while rest > 1:
            binary.append(rest % 2)
            rest = rest//2
        # rest 1 or 0
        binary.append(rest % 2)
        
        # Complete with zeros to fill 32 bits
        binary.extend([0]*(bits-len(binary)))
            

        return binary[::-1] 
    
    def fromBinary(self, bNum, sigDig=4):
        l = len(bNum)
        return sum([bNum[l-1-i]*(2**i) for i in range(l)])/10**sigDig

In [55]:
a = [1,1,1]
a.extend([[0]*5])
a

[1, 1, 1, [0, 0, 0, 0, 0]]

In [56]:
class GeneticAlgorithm:
    Population = []
    Generation = 0
    BestFitness = 0
    BestAngle = 0
    BestGenes = []
    FitnessSum = 0
    
    mutationRate = 0.01
    
    newPopulation = []
    popSize = 0
    eliteSize = 5
        
    def __init__(self, popSize, dnaSize, eliteSize=5):
        self.Generation = 1;
        self.Population = [DNA(dnaSize) for i in range(popSize)]
        self.popSize = popSize
        self.eliteSize = eliteSize
        self.FitnessSum = 0
        print("Random population of {} individuals created.".format(popSize))
        
    def newGeneration(self):
        if self.popSize == 0:
            print("Error: Empty population")
            return
        
        #Sort population by fitness
        self.Population.sort(key=lambda x: x.Fitness, reverse=True)
        #Start creating a new population
        self.newPopulation = []
        for i in range(self.popSize):
            print("Generating new individuals {} / {} ".format(i, self.popSize), end="\r")
            if i < self.eliteSize:
                self.newPopulation.append(self.Population[i])
            else:
                parent1 = self.chooseParent()
                parent2 = self.chooseParent()

                child = parent1.crossover(parent2)

                child.mutate(self.mutationRate)

                self.newPopulation.append(child)
        print("\nDone.")
        # Swap entire lists <:O!
        self.Population, self.newPopulation = self.newPopulation, self.Population
        
        self.Generation += 1
            
    def chooseParent(self):
        threshold = np.random.random() * self.FitnessSum
        #print(threshold)
        
        for i in range(len(self.Population)):
            if self.Population[i].Fitness > threshold:
                return self.Population[i]
            threshold -= abs(self.Population[i].Fitness)
        # Try again
        return self.chooseParent()
    
    def calculateFitness(self):
        model = load_model_from_path(xml_path)
        for i in range(self.popSize):
            # Get angle from gene
            print("Simulating {} / {} ".format(i, self.popSize), end="\r")
            angle = self.Population[i].Genes[0]
            #print("Angle:", angle)
            # Modify mujoco model ramp with that angle
            model.body_quat[1] = [0.25, angle, 0, 0]
            sim = MjSim(model)
            sim.reset()
            for j in range(300):
                sim.step()
            # Get horizontal position of ball
            fitness = sim.data.qpos[1]
            #print("Fitness {}: {}".format(i, fitness))
            self.Population[i].setFitness(fitness)
            self.FitnessSum += abs(fitness)
            # Save best individual
            if fitness > self.BestFitness:
                self.BestFitness = fitness
                self.BestAngle = self.Population[i].Genes[0]
        # Information
        print("\nDone.")
        print("Generation: {} ".format(self.Generation) )
        print("Best fitness: ", self.BestFitness)
        print("Best angle: ", self.BestAngle)

In [57]:
    def bkpcalculateFitness(self):
        sims = []
        for i in range(self.popSize):
            # Get angle from gene
            angle = self.Population[i].Genes[0]
            #print("Angle:", angle)
            # Modify mujoco model ramp with that angle
            model = load_model_from_path(xml_path)
            model.body_quat[1] = [0.25, angle, 0, 0]
            sim = MjSim(model)
            sims.append(sim)

        for i in range(self.popSize):
            sims[i].reset()
            for j in range(1500):
                sims[i].step()
            # Get horizontal position of ball
            fitness = sims[i].data.qpos[1]
            #print("Fitness {}: {}".format(i, fitness))
            self.Population[i].setFitness(fitness)
            self.FitnessSum += abs(fitness)
            # Save best individual
            if fitness > self.BestFitness:
                self.BestFitness = fitness
                self.BestAngle = self.Population[i].Genes[0]

In [58]:
GA = GeneticAlgorithm(popSize=10000, dnaSize=1)

Random population of 10000 individuals created.


In [59]:
def showTopN(n=10):
    print("Top {} individuals:\n".format(n))
    GA.Population.sort(key=lambda x: x.Fitness, reverse=True)
    for i in range(n):
        print("{}: Fitness: {}\t\tAngle: {}".format(i+1, GA.Population[i].Fitness, GA.Population[i].Genes))
    
showTopN()

Top 10 individuals:

1: Fitness: 0.0		Angle: [-0.250919762305275]
2: Fitness: 0.0		Angle: [0.9014286128198323]
3: Fitness: 0.0		Angle: [0.4639878836228102]
4: Fitness: 0.0		Angle: [0.1973169683940732]
5: Fitness: 0.0		Angle: [-0.687962719115127]
6: Fitness: 0.0		Angle: [-0.6880109593275947]
7: Fitness: 0.0		Angle: [-0.8838327756636011]
8: Fitness: 0.0		Angle: [0.7323522915498704]
9: Fitness: 0.0		Angle: [0.2022300234864176]
10: Fitness: 0.0		Angle: [0.416145155592091]


In [60]:
while True:
    GA.calculateFitness()
    showTopN()
    GA.newGeneration()

Done.ating 9999 / 10000 
Generation: 1 
Best fitness:  3.906839127071122
Best angle:  -0.24360730094356198
Top 10 individuals:

1: Fitness: 3.906839127071122		Angle: [-0.24360730094356198]
2: Fitness: 3.9062128038240327		Angle: [0.2566942350988888]
3: Fitness: 3.9062108319991524		Angle: [0.2494013388132159]
4: Fitness: 3.906068746580877		Angle: [0.24932713770852755]
5: Fitness: 3.905946431652785		Angle: [0.2564131941860879]
6: Fitness: 3.905907703348379		Angle: [-0.24336878566948106]
7: Fitness: 3.9059075900935385		Angle: [-0.24393787553360968]
8: Fitness: 3.905818426876361		Angle: [0.2512998258829062]
9: Fitness: 3.9058009555284356		Angle: [-0.2502588409525919]
10: Fitness: 3.9057913025811897		Angle: [-0.24395010113119597]
Done.ating new individuals 9999 / 10000 
Done.ating 9999 / 10000 
Generation: 2 
Best fitness:  3.907193004998579
Best angle:  0.2566
Top 10 individuals:

1: Fitness: 3.907193004998579		Angle: [0.2566]
2: Fitness: 3.907058133557159		Angle: [0.2573]
3: Fitness: 3.907

KeyboardInterrupt: 

In [187]:
GA.calculateFitness()

Angle: 0.07845638134226596
Angle: 0.07845638134226596
Angle: 0.695974206093698
Angle: 0.2708322512620742
Angle: 0.32295647294124596
Angle: 0.695974206093698
Angle: 0.07845638134226596
Angle: 0.02535074341545751
Angle: 0.07845638134226596
Angle: 0.2708322512620742
Fitness 0: 26.03775248178308
Fitness 1: 26.03775248178308
Fitness 2: -27.161008981875824
Fitness 3: -9.957004364745869
Fitness 4: -17.759207430192546
Fitness 5: -27.161008981875824
Fitness 6: 26.03775248178308
Fitness 7: 8.685154796774992
Fitness 8: 26.03775248178308
Fitness 9: -9.957004364745869


In [188]:
GA.newGeneration()

Generation:  3
Best fitness:  26.03775248178308


In [189]:
GA.calculateFitness()

Angle: 0.07845638134226596
Angle: 0.02535074341545751
Angle: 0.07845638134226596
Angle: 0.07845638134226596
Angle: 0.07845638134226596
Angle: 0.07845638134226596
Angle: 0.07845638134226596
Angle: 0.02535074341545751
Angle: 0.07845638134226596
Angle: 0.07845638134226596
Fitness 0: 26.03775248178308
Fitness 1: 8.685154796774992
Fitness 2: 26.03775248178308
Fitness 3: 26.03775248178308
Fitness 4: 26.03775248178308
Fitness 5: 26.03775248178308
Fitness 6: 26.03775248178308
Fitness 7: 8.685154796774992
Fitness 8: 26.03775248178308
Fitness 9: 26.03775248178308


In [190]:
GA.newGeneration()

Generation:  4
Best fitness:  26.03775248178308


In [8]:

        

sim.reset()
print("Initial Y position:\t", sim.data.qpos[1])
for i in range(1500):
    #if i%100==0:
        #print(sim.data.qvel[1])
    sim.step()
    
    #viewer.render()

print("Final Y position:\t", sim.data.qpos[1])

Initial Y position:	 0.0
Final Y position:	 8.559659011576974


In [9]:
#model.mj_name2id("main-plane")

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [11]:
#MUJOCO_GL= environment variable to "glfw", "egl", or "osmesa", respectively.
#MUJOCO_GL="osmesa"

In [12]:
model.body_quat[1] = [0.25, 0.7, 0, 0]

sim = MjSim(model)

In [13]:
import gym

#env = gym.make('Humanoid-v2')
#env.reset()
sim.reset()
viewer = MjViewer(sim)#(sim)
for i in range(15000):
    #plt.imshow(viewer)
    viewer.render()
    sim.step()
viewer.render()
#viewer = None

Creating window glfw


No traceback available to show.


SystemExit: 0

/home/jack/anaconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
model.jnt_axis

In [14]:

%tb

SystemExit: 0